```{contents}
```
## Hallucination Prevention and Control


A **hallucination** occurs when an LLM generates **confident but incorrect, fabricated, or unverifiable information** that is **not grounded in the provided data or reality**.

> Hallucinations are not bugs — they are a natural outcome of probabilistic language generation.

Prevention and control require **system-level design**, not just better prompts.

---

### Why Hallucinations Happen

#### Root Causes

1. **No grounding data**
2. **Ambiguous or underspecified prompts**
3. **Excessive context or noisy context**
4. **High temperature / randomness**
5. **Training bias toward fluent completion**
6. **Model guessing to satisfy the prompt**

---

### Where Hallucination Control Fits

```
User Input
  ↓
Input Guardrails
  ↓
Retrieval (Grounding)
  ↓
Context Selection
  ↓
Prompt Constraints
  ↓
LLM
  ↓
Output Validation
```

Hallucination control must exist **at every stage**.

---

### Layered Hallucination Prevention Strategy

#### Layer 1: Grounding with Retrieval (Primary Defense)

**RAG (Retrieval-Augmented Generation)** forces the model to answer **from real documents**.

```
Question → Retrieve Context → Answer from Context
```

Key rule:

> If it’s not in the retrieved context, the model should say *“I don’t know”*.

---

#### Layer 2: Strong Prompt Constraints

#### Example Prompt Guardrail

```text
Answer only using the provided context.
If the answer is not present, say "I don’t know".
Do not use outside knowledge.
```

This reduces, but does not eliminate hallucinations.

---

#### Layer 3: Context Quality Control

### Techniques

* Proper **chunking**
* **Chunk overlap**
* **Top-k limits**
* **MMR (diversity)**
* **Reranking (precision)**

Bad context = hallucinated answers.

---

#### Layer 4: Similarity Thresholding

Reject weak retrievals.

```text
If similarity score < threshold → do not answer
```

Production pattern:

```
Low confidence → Ask clarifying question or escalate
```

---

#### Layer 5: Output Structure Enforcement

Structured outputs reduce creative guessing.

### Example

```python
llm.with_structured_output(Schema)
```

Benefits:

* Forces explicit fields
* Prevents free-form fabrication
* Enables validation

---

#### Layer 6: Output Validation & Guardrails

**Validation Examples**

* Schema validation (Pydantic)
* Allowed values (enums)
* Numeric range checks
* Regex checks

Invalid outputs are:

* Rejected
* Re-prompted
* Escalated

---

#### Layer 7: Temperature and Sampling Control

**Recommended Settings**

| Task             | Temperature |
| ---------------- | ----------- |
| Fact-based QA    | 0.0–0.2     |
| RAG answers      | 0.0         |
| Creative writing | 0.7+        |

Lower temperature = fewer hallucinations.

---

#### Layer 8: Context Window Management

Too much context causes:

* Contradictions
* Confusion
* Fabrication

Use:

* Reranking
* Context compression
* Summarization
* Memory pruning

**Relevant context beats maximum context.**

---

#### Layer 9: Source Attribution (Detection Tool)

Hallucinations become visible when:

* Sources are missing
* Sources don’t support claims

Production rule:

> No source → low confidence answer.

---

#### Layer 10: Human-in-the-Loop (Fail-Safe)

For critical systems:

* Low confidence → escalate to human
* Hallucination risk → require approval

Used in:

* Legal
* Medical
* Compliance
* IT incident response

---

### Hallucination Control by Stage

#### Input Stage

* Prompt injection detection
* Input sanitization
* Clarification requests

#### Retrieval Stage

* Hybrid search
* Reranking
* Similarity thresholds

#### Generation Stage

* Low temperature
* Constrained prompts
* Tool-only answers

#### Output Stage

* Validation
* Schema enforcement
* Source checks

---

### What Does NOT Prevent Hallucinations

❌ Prompt engineering alone
❌ Larger context windows
❌ More retrieved documents
❌ Asking the model to “be accurate”
❌ Trusting confidence in tone

---

### Common Production Mistakes

#### Over-retrieving context

❌ Increases hallucinations

#### No similarity threshold

❌ Model answers without evidence

#### Trusting LLM citations

❌ Citations can be hallucinated

#### No fallback logic

❌ Users get wrong answers instead of “unknown”

---

### Production-Grade Hallucination Control Checklist

* ✅ RAG with high-quality data
* ✅ Hybrid retrieval + reranking
* ✅ Similarity thresholds
* ✅ Strong system prompts
* ✅ Structured output validation
* ✅ Source attribution
* ✅ Human escalation path
* ✅ Observability & logging

---

### Measuring Hallucinations (Detection)

Indicators:

* Answer without sources
* Low retrieval scores
* Contradictory statements
* Overconfident language

Metrics:

* Answer groundedness
* Retrieval confidence
* Source coverage ratio

---

### Interview-Ready Summary

> “Hallucination prevention is a layered system design problem. It combines grounding via retrieval, prompt constraints, context control, low-temperature generation, structured outputs, validation, source attribution, and human-in-the-loop safeguards.”

---

### Rule of Thumb

* **No grounding → hallucinations**
* **More context ≠ safer answers**
* **Structure reduces guessing**
* **If accuracy matters, validate everything**